In [1]:
import os
import re
import numpy as np
from tqdm import tqdm
import pandas as pd

In [2]:
np.random.seed(42)

In [3]:
MAX_LEN = 60
CLASS_NUMBER = 6
BATCH_SIZE = 10
NUM_EPOCHS = 100
VALID_SPLIT = 0.2
BERT_CKPT = './data_out/'
DATA_IN_PATH = '../metadata/'
DATA_OUT_PATH = './data_out/'

In [77]:
lyrics2 = pd.read_csv(DATA_IN_PATH+'Song_Lyrics2.csv')

In [78]:
lyrics2

,IDX,SONG_ID,LYRICS
0,14,2466,나도 알지 못한날 굳이 알려고 들지마 \n보여줄 수 없는 내 마음만 안타까울 뿐 \...
1,163,5112,제각기 있기 였기 때문이기 \n사랑했기 때문이기 \n하루지기 이야기 이야기 \n잊지...
2,316,7712,당신은 모르실 거야\n얼마나 사랑했는지\n세월이 흘러가면은\n그때서 뉘우칠 거야\n...
3,464,9270,이렇게 멋진 파란 하늘 위로 \n날으는 마법 융단을 타고 \n이렇게 멋진 푸른 세상...
4,486,9302,\N
...,...,...,...
3740,348575,35353251,그대 아무런 말도 하지 마요\n이 맘은 여전히 그대로예요\n따가운 햇살 그 아래 우...
3741,348722,35353557,대충 모자를 눌러 쓰고서\n부은 눈을 가리고 나와\n오늘부터는 울어도\n나 때문에 ...
3742,349518,35380072,지금 시각 열한 시 이십팔 분\n물론 아니지 아침\n하루 일과 다 마치구\n우연히 ...
3743,349569,35389041,"Come and, talk to me baby\n뭐든 콕 찝어 말을 해줘\n사소한 ..."


In [79]:
lyrics2.isnull().sum()

IDX        0
SONG_ID    0
LYRICS     5
dtype: int64

In [80]:
lyrics2 = lyrics2.dropna()

In [81]:
def preprocessing(x):
    temp = x.split('\n')
    temp = ' '.join(temp)
    return temp

In [82]:
lyrics2['Edited_LYRICS'] = lyrics2['LYRICS'].apply(preprocessing)

c:\project_huiyeol\project\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [83]:
def slicing(x):
    x = x.split()
    res = []
    for i in range(0, len(x)+1, 120):
        if len(x)-i < 120:
            temp = x[i:]
            res.append(temp)
            break
        temp = x[i:i+120]
        res.append(temp)
    return res

In [84]:
lyrics2['slice'] = lyrics2['Edited_LYRICS'].apply(slicing)

c:\project_huiyeol\project\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [85]:
length = lyrics2['Edited_LYRICS'].apply(lambda x:len(x))
length.mean()

808.4994652406417

In [86]:
lyrics2['Sliced_LYRICS'] = ''

c:\project_huiyeol\project\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [87]:
lyrics2

,IDX,SONG_ID,LYRICS,Edited_LYRICS,slice,Sliced_LYRICS
0,14,2466,나도 알지 못한날 굳이 알려고 들지마 \n보여줄 수 없는 내 마음만 안타까울 뿐 \...,나도 알지 못한날 굳이 알려고 들지마 보여줄 수 없는 내 마음만 안타까울 뿐 이...,"[[나도, 알지, 못한날, 굳이, 알려고, 들지마, 보여줄, 수, 없는, 내, 마음...",
1,163,5112,제각기 있기 였기 때문이기 \n사랑했기 때문이기 \n하루지기 이야기 이야기 \n잊지...,제각기 있기 였기 때문이기 사랑했기 때문이기 하루지기 이야기 이야기 잊지않기 ...,"[[제각기, 있기, 였기, 때문이기, 사랑했기, 때문이기, 하루지기, 이야기, 이야...",
2,316,7712,당신은 모르실 거야\n얼마나 사랑했는지\n세월이 흘러가면은\n그때서 뉘우칠 거야\n...,당신은 모르실 거야 얼마나 사랑했는지 세월이 흘러가면은 그때서 뉘우칠 거야 When...,"[[당신은, 모르실, 거야, 얼마나, 사랑했는지, 세월이, 흘러가면은, 그때서, 뉘...",
3,464,9270,이렇게 멋진 파란 하늘 위로 \n날으는 마법 융단을 타고 \n이렇게 멋진 푸른 세상...,이렇게 멋진 파란 하늘 위로 날으는 마법 융단을 타고 이렇게 멋진 푸른 세상 속...,"[[이렇게, 멋진, 파란, 하늘, 위로, 날으는, 마법, 융단을, 타고, 이렇게, ...",
4,486,9302,\N,\N,[[\N]],
...,...,...,...,...,...,...
3740,348575,35353251,그대 아무런 말도 하지 마요\n이 맘은 여전히 그대로예요\n따가운 햇살 그 아래 우...,그대 아무런 말도 하지 마요 이 맘은 여전히 그대로예요 따가운 햇살 그 아래 우리 ...,"[[그대, 아무런, 말도, 하지, 마요, 이, 맘은, 여전히, 그대로예요, 따가운,...",
3741,348722,35353557,대충 모자를 눌러 쓰고서\n부은 눈을 가리고 나와\n오늘부터는 울어도\n나 때문에 ...,대충 모자를 눌러 쓰고서 부은 눈을 가리고 나와 오늘부터는 울어도 나 때문에 울어 ...,"[[대충, 모자를, 눌러, 쓰고서, 부은, 눈을, 가리고, 나와, 오늘부터는, 울어...",
3742,349518,35380072,지금 시각 열한 시 이십팔 분\n물론 아니지 아침\n하루 일과 다 마치구\n우연히 ...,지금 시각 열한 시 이십팔 분 물론 아니지 아침 하루 일과 다 마치구 우연히 접한 ...,"[[지금, 시각, 열한, 시, 이십팔, 분, 물론, 아니지, 아침, 하루, 일과, ...",
3743,349569,35389041,"Come and, talk to me baby\n뭐든 콕 찝어 말을 해줘\n사소한 ...","Come and, talk to me baby 뭐든 콕 찝어 말을 해줘 사소한 것까...","[[Come, and,, talk, to, me, baby, 뭐든, 콕, 찝어, 말...",


In [88]:
new_data = pd.DataFrame(lyrics2.iloc[0][:]).T
for idx in tqdm(lyrics2.index):
    try:
        for i in range(len(lyrics2.loc[idx]['slice'])):
            temp = lyrics2.loc[idx]['slice'][i]
            txt = ' '.join(temp)
            lyrics2.loc[idx, 'Sliced_LYRICS'] = txt
            new_data = pd.concat([new_data, pd.DataFrame(lyrics2.loc[idx]).T])
    except:
        continue

  0%|          | 0/3740 [00:00<?, ?it/s]c:\project_huiyeol\project\venv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
100%|██████████| 3740/3740 [00:13<00:00, 285.22it/s]


In [89]:
new_data.reset_index(drop=True, inplace=True)

In [123]:
new_data[new_data['Sliced_LYRICS']=='\\N']

,IDX,SONG_ID,LYRICS,Edited_LYRICS,slice,Sliced_LYRICS
8,486,9302,\N,\N,[[\N]],\N
61,3581,50021,\N,\N,[[\N]],\N
200,15531,335616,\N,\N,[[\N]],\N
202,17099,384136,\N,\N,[[\N]],\N
261,23079,503486,\N,\N,[[\N]],\N
...,...,...,...,...,...,...
8459,341930,106756721,\N,\N,[[\N]],\N
8460,342440,106785314,\N,\N,[[\N]],\N
8461,342735,106809790,\N,\N,[[\N]],\N
8462,343606,106857266,\N,\N,[[\N]],\N


In [90]:
new_data = new_data.drop(new_data[new_data['Sliced_LYRICS']==''].index)

In [94]:
new_data = new_data.drop(new_data[new_data['Sliced_LYRICS']=='None'].index)

In [124]:
new_data = new_data.drop(new_data[new_data['Sliced_LYRICS']=='\\N'].index)

In [149]:
lyrics = new_data[['SONG_ID', 'Sliced_LYRICS']]

In [150]:
lyrics['대분류1']=''
lyrics['대분류2']=''

c:\project_huiyeol\project\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\project_huiyeol\project\venv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [151]:
from sklearn.model_selection import train_test_split

data_train, data_valid = train_test_split(lyrics, test_size=VALID_SPLIT, random_state = 42)

In [152]:
data_train.sort_index(inplace=True)
data_valid.sort_index(inplace=True)

In [153]:
data_train.to_csv(DATA_OUT_PATH+'train_slicedLyrics.csv', encoding='utf-8-sig')

In [154]:
data_valid.to_csv(DATA_OUT_PATH+'valid_slicedLyrics.csv', encoding='utf-8-sig')